In [4]:
%run imports_and_functions.ipynb

from dbfread import DBF

C:\Users\Jason\AppData\Local\conda\conda\envs\DSI-6\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [7]:
# income data files come in .dbf, must install this to read from python

# !pip install dbfread

### Creating Final Businesses Dataframe within City Radius

In [9]:
df = pd.read_csv('SeattleBusinesses.csv', sep=';', compression='gzip')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17028 entries, 0 to 17027
Data columns (total 12 columns):
Unnamed: 0       17028 non-null int64
alias            17028 non-null object
categories       17028 non-null object
coordinates      17028 non-null object
distance         17028 non-null float64
latitude         16966 non-null float64
longitude        16966 non-null float64
main_category    17028 non-null object
name             17028 non-null object
price            5866 non-null object
rating           17028 non-null float64
review_count     17028 non-null int64
dtypes: float64(4), int64(2), object(6)
memory usage: 1.6+ MB


In [10]:
def dropduplicate_yelp_entries(df) :
    dups = df[df[['latitude','longitude','alias']].duplicated(keep=False)].sort_values('latitude')
    mask = dups[['latitude','longitude','alias']].duplicated(keep='first')
#     print(dups)
    for index in dups[mask].index :
        dupset = dups[(dups['alias']==dups.loc[index,'alias']) & (dups['longitude']==dups.loc[index,'longitude'])].sort_values(
            'review_count', ascending=False)
        dropindex = dupset.iloc[1:,0].index
        df.drop(dropindex, axis=0, inplace=True)

def getPriceRange(string) :
    regex = re.compile("RestaurantsPriceRange2\\': \\'(\d)")
    try :
        match = regex.search(string)
        return int(match.groups(0)[0])
    except : return 0

def get_main_category(catstring) :
    string = ''
    try :
        for cat in main_categories :
            if cat in catstring : string = string + cat + ', '
    except : pass
    return string[0:(len(string)-2)]

def convertpricecolumn(pricesymbol) :
    try : num = len(pricesymbol)
    except : num = 0
    return num

In [11]:
print(df.shape)
dropduplicate_yelp_entries(df)
df.shape

(17028, 12)


(17028, 12)

In [12]:
# Still quite low percentage that have price field, but in new yelp data it's no longer found in
# attribute field under 'PriceRange2'
# df['price'] = df['attributes'].map(getPriceRange)
df['price'] = df['price'].map(convertpricecolumn)
df['price'].value_counts()

0    11162
2     3271
1     1849
3      627
4      119
Name: price, dtype: int64

In [14]:
# SET FOR EACH CITY: maxradius, testlat, testlong
maxradius = 6000
testlat = 47.592989
testlong = -122.336087 # Seattle, WA

In [15]:
df['distance2'] = calcDistances(testlat, testlong, df)

df[df['distance2']<maxradius].shape

(12516, 13)

In [16]:
# yelp calculated distances seem within 30 meters of python geo-module calculations
# need to use these ones because of 'extra_squares' search where yelp returns distance to these
# new auxilary centers, not the true city center
df['distance'] = df['distance2']

In [33]:
# df[df[['latitude','longitude']].duplicated(keep=False)].sort_values('latitude')

In [17]:
rowstodrop = df[(df['distance']>maxradius) | df['latitude'].isnull() | df['longitude'].isnull()].index

df.drop(rowstodrop, axis=0, inplace=True)
df.shape

(12516, 13)

In [18]:
df['lat_index'] = round(df['latitude'], 3)
df['long_index'] = round(df['longitude'], 4)

df = df.set_index(['lat_index','long_index'], drop=False).sort_index()
df = df.sort_index(level = [0,1])

In [19]:
df.tail()

Unnamed: 0                            alias  \
lat_index long_index                                                
47.646    -122.3257        10225            lindquist-law-seattle   
          -122.3250        18815           eastlake-flats-seattle   
          -122.3240        16942        wellbelly-healing-seattle   
          -122.3236        10222  lisa-k-clark-law-office-seattle   
          -122.3236        10223         bromet-kenneth-f-seattle   

                                                             categories  \
lat_index long_index                                                      
47.646    -122.3257   [{'alias': 'personal_injury', 'title': 'Person...   
          -122.3250    [{'alias': 'apartments', 'title': 'Apartments'}]   
          -122.3240   [{'alias': 'massage_therapy', 'title': 'Massag...   
          -122.3236   [{'alias': 'divorce', 'title': 'Divorce & Fami...   
          -122.3236          [{'alias': 'lawyers', 'title': 'Lawyers'}]   

                                                            coordinates  \
lat_index long_index                                                      
47.646    -122.3257   {'latitude': 47.6463656, 'longitude': -122.325...   
          -122.3250     {'latitude': 47.64615, 'longitude': -122.32505}   
          -122.3240   {'latitude': 47.6459199, 'longitude': -122.32398}   
          -122.3236      {'latitude': 47.6458, 'longitude': -122.32359}   
          -122.3236      {'latitude': 47.6458, 'longitude': -122.32359}   

                         distance   latitude   longitude main_category  \
lat_index long_index                                                     
47.646    -122.3257   5985.258800  47.646366 -122.325746  professional   
          -122.3250   5968.542388  47.646150 -122.325050  homeservices   
          -122.3240   5954.970197  47.645920 -122.323980        health   
          -122.3236   5946.357577  47.645800 -122.323590  professional   
          -122.3236   5946.357577  47.645800 -122.323590  professional   

                                         name  price  rating  review_count  \
lat_index long_index                                                         
47.646    -122.3257             Lindquist Law      0     5.0             4   
          -122.3250            Eastlake Flats      0     1.0             3   
          -122.3240         Wellbelly Healing      0     5.0             1   
          -122.3236   Lisa K Clark Law Office      0     1.0             1   
          -122.3236          Bromet Kenneth F      0     5.0             1   

                        distance2  lat_index  long_index  
lat_index long_index                                      
47.646    -122.3257   5985.258800     47.646   -122.3257  
          -122.3250   5968.542388     47.646   -122.3250  
          -122.3240   5954.970197     47.646   -122.3240  
          -122.3236   5946.357577     47.646   -122.3236  
          -122.3236   5946.357577     47.646   -122.3236

In [20]:
cat_alias_dict = json.load(open('cat_alias_dict.json','r'))

def categories_to_alias2(dfrow) :
    catstring = dfrow['categories']
    if type(catstring) != str : return str(dfrow['price']) + '_other'
    rawlis = list(set(catstring.split(', ')))
    lis = []
    for item in rawlis :
        if 'alias' in item :
            alias = item.split(':')[1].strip().strip("'")
            lis += [alias]
#     print(lis)
    lis = list(map(get_alias2,lis))
    lis = [item for item in lis if item != '_other']
    lis += ['_' + dfrow['main_category']]
    # some new categories are not in category dictionary, and each row already has main category
    # so am taking out all instances where business category could not be found
    lis = set(map((lambda x : str(dfrow['price']) + x),lis))
    return ' '.join(lis)

def get_alias2(cat) :
    try :
        alias = '_' + str(cat_alias_dict[map_category(cat_alias_dict[cat])])
    except : alias = '_other'
    return alias

def categories_to_alias(dfrow) :
    catstring = dfrow['categories']
    if type(catstring) != str : return str(dfrow['price']) + '_other'
    lis = list(set(catstring.split(', ')))
    lis = map(get_alias,lis)
    lis = set(map((lambda x : str(dfrow['price']) + x),lis))
    return ' '.join(lis)

def get_alias(cat) :
    try :
        alias = '_' + str(cat_alias_dict[map_category(cat.lower())])
    except : alias = '_other'
    return alias

## Function to map titles to macro-categories if not in top 150

dict_2A = json.load(open('dict_2A.json','r'))
dict_2B = json.load(open('dict_2B.json','r'))

def map_category(category):
    if category not in top_148:
        if category in dict_2B.keys():
            title = dict_2B[category]
        else:
            title = category
        if title in dict_2A.keys():
            title_2 = dict_2A[title]
            return(title_2)
        else:
            return(title)
    else:
        return(category)

In [21]:
df['alias_cats'] = df.apply(categories_to_alias2, axis=1)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 12516 entries, (47.539, -122.3414) to (47.646, -122.3236)
Data columns (total 16 columns):
Unnamed: 0       12516 non-null int64
alias            12516 non-null object
categories       12516 non-null object
coordinates      12516 non-null object
distance         12516 non-null float64
latitude         12516 non-null float64
longitude        12516 non-null float64
main_category    12516 non-null object
name             12516 non-null object
price            12516 non-null int64
rating           12516 non-null float64
review_count     12516 non-null int64
distance2        12516 non-null float64
lat_index        12516 non-null float64
long_index       12516 non-null float64
alias_cats       12516 non-null object
dtypes: float64(7), int64(3), object(6)
memory usage: 1.6+ MB


In [23]:
df.head()

Unnamed: 0                                      alias  \
lat_index long_index                                                          
47.539    -122.3414        20055  pacific-plumbing-supply-company-seattle-2   
47.540    -122.3383        10220               dandylines-show-room-seattle   
          -122.3228        14764            calozzis-cheesesteaks-seattle-3   
47.541    -122.3377        21396                barron-collection-seattle-2   
          -122.3277         1740                 alpine-auto-center-seattle   

                                                             categories  \
lat_index long_index                                                      
47.539    -122.3414        [{'alias': 'plumbing', 'title': 'Plumbing'}]   
47.540    -122.3383   [{'alias': 'homedecor', 'title': 'Home Decor'}...   
          -122.3228   [{'alias': 'cheesesteaks', 'title': 'Cheeseste...   
47.541    -122.3377     [{'alias': 'giftshops', 'title': 'Gift Shops'}]   
          -122.3277   [{'alias': 'bodyshops', 'title': 'Body Shops'}...   

                                                            coordinates  \
lat_index long_index                                                      
47.539    -122.3414     {'latitude': 47.53926, 'longitude': -122.34143}   
47.540    -122.3383   {'latitude': 47.5404648292527, 'longitude': -1...   
          -122.3228   {'latitude': 47.540249, 'longitude': -122.3228...   
47.541    -122.3377   {'latitude': 47.5410907, 'longitude': -122.337...   
          -122.3277   {'latitude': 47.5410474, 'longitude': -122.327...   

                         distance   latitude   longitude main_category  \
lat_index long_index                                                     
47.539    -122.3414   5987.205780  47.539260 -122.341430  homeservices   
47.540    -122.3383   5842.077766  47.540465 -122.338285  professional   
          -122.3228   5947.788414  47.540249 -122.322845   restaurants   
47.541    -122.3377   5771.500750  47.541091 -122.337746      shopping   
          -122.3277   5809.509324  47.541047 -122.327680    automotive   

                                                 name  price  rating  \
lat_index long_index                                                   
47.539    -122.3414   Pacific Plumbing Supply Company      0     4.0   
47.540    -122.3383              DandyLines Show Room      1     5.0   
          -122.3228            Calozzi's Cheesesteaks      2     4.5   
47.541    -122.3377                 Barron Collection      0     5.0   
          -122.3277                Alpine Auto Center      0     4.0   

                      review_count    distance2  lat_index  long_index  \
lat_index long_index                                                     
47.539    -122.3414              5  5987.205780     47.539   -122.3414   
47.540    -122.3383              1  5842.077766     47.540   -122.3383   
          -122.3228            234  5947.788414     47.540   -122.3228   
47.541    -122.3377              1  5771.500750     47.541   -122.3377   
          -122.3277             16  5809.509324     47.541   -122.3277   

                                                        alias_cats  
lat_index long_index                                                
47.539    -122.3414                      0_homeservices 0_plumbing  
47.540    -122.3383       1_professional 1_homedecor 1_accessories  
          -122.3228                     2_restaurants 2_sandwiches  
47.541    -122.3377                                     0_shopping  
          -122.3277   0_automotive 0_bodyshops 0_auto 0_autorepair

In [42]:

df.to_csv('SeattleBusinessesFinal.csv',sep=';',header=True,index=True,compression='gzip')

In [41]:
df['alias_cats'].map((lambda x : str(len(x))+x)).value_counts()

80_health                                                 772
140_professional                                          613
140_homeservices                                          384
150_localservices                                         370
80_active                                                 309
140_hotelstravel                                          283
270_homeservices 0_apartments                             244
150_eventservices                                         243
250_realestate 0_apartments                               222
60_arts                                                   182
190_automotive 0_auto                                     182
310_realestateagents 0_realestate                         168
102_shopping                                              164
280_contractors 0_homeservices                            163
110_education                                             162
100_shopping                                              126
270_fina

### Census Block Population and Income to X_train Coordinates DataFrame

In [26]:
table = DBF('tl_2014_53_tabblock10.dbf')
type(table)

dbfread.dbf.DBF

In [27]:
df_coords = pd.DataFrame(iter(table))
df_coords.head()

,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE10,GEOID10,NAME10,MTFCC10,UR10,UACE10,UATYP10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10
0,53,073,010100,3147,530730101003147,Block 3147,G5040,R,,,S,0,7503,+48.8311355,-122.1424270
1,53,073,010100,4003,530730101004003,Block 4003,G5040,R,,,S,0,12106,+48.8324631,-122.1557965
2,53,073,010100,4182,530730101004182,Block 4182,G5040,R,,,S,0,87185,+48.6698268,-122.1447266
3,53,073,011000,1018,530730110001018,Block 1018,G5040,R,,,S,0,120126,+48.9779153,-123.0668955
4,53,073,940000,1027,530739400001027,Block 1027,G5040,R,,,S,0,81726,+48.7731088,-122.5998590


In [28]:
df_coords = df_coords[['GEOID10','INTPTLAT10','INTPTLON10','UR10','ALAND10','AWATER10']]
# df_coords.to_csv('SeattleCoords.csv',sep=';',header=True,index=True,compression='gzip')

In [29]:
def cut(geoid) :
    index = str(geoid)
    return int(index[0:(len(index)-3)])

def matchincome(geoid) :
    try : return blockgroupincome.loc[geoid,'income']
    except : return 0

def matchpop(geoid) :
    try : return blockgrouppop.loc[geoid,'population']
    except : return 0

In [30]:
table = DBF('tabblock2010_53_pophu.dbf')

populations = pd.DataFrame(iter(table))
populations.head()

,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE,BLOCKID10,PARTFLG,HOUSING10,POP10
0,53,001,950100,1042,530019501001042,N,1,5
1,53,001,950200,3154,530019502003154,N,0,0
2,53,001,950200,3156,530019502003156,N,2,9
3,53,001,950300,3030,530019503003030,N,0,0
4,53,001,950300,3042,530019503003042,N,2,11


In [31]:
populations = populations[['BLOCKID10','POP10']]

In [32]:
populations.columns = ['GEO.id2','population']
populations['blockgroup'] = populations['GEO.id2'].map(cut)
populations['GEO.id2'] = populations['GEO.id2'].map(int)
populations.set_index(['GEO.id2'],inplace=True,drop=False)
populations.head()

,GEO.id2,population,blockgroup
GEO.id2,,,
530019501001042,530019501001042,5,530019501001
530019502003154,530019502003154,0,530019502003
530019502003156,530019502003156,9,530019502003
530019503003030,530019503003030,0,530019503003
530019503003042,530019503003042,11,530019503003


In [33]:
populations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 195574 entries, 530019501001042 to 530770028023046
Data columns (total 3 columns):
GEO.id2       195574 non-null int64
population    195574 non-null int64
blockgroup    195574 non-null int64
dtypes: int64(3)
memory usage: 6.0 MB


In [34]:
blockgroupincome = pd.read_csv('Seattle.csv')
blockgroupincome.head()

,GEO.id,GEO.id2,GEO.display-label,HD01_VD01,HD02_VD01
0,1500000US530330001001,530330001001,"Block Group 1, Census Tract 1, King County, Wa...",82066.0,24420.0
1,1500000US530330001002,530330001002,"Block Group 2, Census Tract 1, King County, Wa...",29449.0,6510.0
2,1500000US530330001003,530330001003,"Block Group 3, Census Tract 1, King County, Wa...",32027.0,7036.0
3,1500000US530330001004,530330001004,"Block Group 4, Census Tract 1, King County, Wa...",19481.0,3596.0
4,1500000US530330001005,530330001005,"Block Group 5, Census Tract 1, King County, Wa...",33317.0,5586.0


In [35]:
blockgroupincome.drop(0, axis=0, inplace=True)
blockgroupincome['GEO.id2'] = blockgroupincome['GEO.id2'].map(int)
blockgroupincome.set_index('GEO.id2', inplace=True)
blockgroupincome.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1421 entries, 530330001002 to 530339901000
Data columns (total 4 columns):
GEO.id               1421 non-null object
GEO.display-label    1421 non-null object
HD01_VD01            1419 non-null float64
HD02_VD01            1419 non-null float64
dtypes: float64(2), object(2)
memory usage: 55.5+ KB


In [36]:
blockgroupincome.columns = ['notused','notused2','income','errorbar']
blockgroupincome.head()

,notused,notused2,income,errorbar
GEO.id2,,,,
530330001002,1500000US530330001002,"Block Group 2, Census Tract 1, King County, Wa...",29449.0,6510.0
530330001003,1500000US530330001003,"Block Group 3, Census Tract 1, King County, Wa...",32027.0,7036.0
530330001004,1500000US530330001004,"Block Group 4, Census Tract 1, King County, Wa...",19481.0,3596.0
530330001005,1500000US530330001005,"Block Group 5, Census Tract 1, King County, Wa...",33317.0,5586.0
530330002001,1500000US530330002001,"Block Group 1, Census Tract 2, King County, Wa...",41861.0,7882.0


In [37]:
populations['income'] = populations['blockgroup'].map(matchincome)
populations['income'].value_counts()

0.0         159765
28781.0        546
61652.0        263
43314.0        250
53975.0        226
26587.0        197
47170.0        170
8730.0         170
54694.0        149
53916.0        133
45565.0        129
27155.0        123
46691.0        121
49280.0        121
45518.0        109
47327.0        106
31184.0        101
32598.0         98
50778.0         97
34511.0         93
42507.0         91
50094.0         90
32102.0         90
38095.0         89
35822.0         87
41464.0         86
56123.0         85
86364.0         85
49322.0         84
65926.0         83
47333.0         83
61587.0         82
53636.0         82
49083.0         81
53649.0         81
47510.0         80
22951.0         80
47804.0         77
37625.0         77
57510.0         76
56916.0         74
24119.0         74
36742.0         74
75025.0         73
26781.0         73
35578.0         71
33795.0         71
50147.0         70
75693.0         69
56119.0         69
29898.0         69
50911.0         68
96399.0     

In [14]:
# blockgrouppop = blockgroupincome

# Population by census block can replace block group population column
# populations['blockgrouppop'] = populations['blockgroup'].map(matchpop)
# populations['blockgrouppop'].value_counts()

C:\Users\Jason\AppData\Local\conda\conda\envs\DSI-6\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


0         168987
17326        787
14746        753
78229        661
18185        639
22581        622
31203        573
3880         531
33868        487
-            419
32150        416
20328        392
18930        301
33053        282
21180        278
19604        269
21896        260
47731        256
21618        239
18767        228
30785        225
16644        203
24263        196
30453        182
18186        172
31979        171
46435        171
12395        171
15586        160
26170        157
39747        154
43373        152
24003        150
29197        150
26288        145
25530        138
12132        134
23799        133
112222       126
20814        122
17091        119
12895        116
43901        114
35941        113
33372        112
35455        111
28170        111
20964        109
25307        108
38521        107
21718        105
36011        104
27142        101
32154        100
15428        100
54499         98
38691         98
18003         97
34027         

In [38]:
populations.head()

,GEO.id2,population,blockgroup,income
GEO.id2,,,,
530019501001042,530019501001042,5,530019501001,0.0
530019502003154,530019502003154,0,530019502003,0.0
530019502003156,530019502003156,9,530019502003,0.0
530019503003030,530019503003030,0,530019503003,0.0
530019503003042,530019503003042,11,530019503003,0.0


### X_train Coordinates DataFrame

In [39]:
features = ['distance','urban','latitude','longitude','blockgroup','income',
                'blockgrouppop','population','landarea','waterarea','block']

In [40]:
def blocksdropna(df) :
    df.drop(df[df['income']==0].index, inplace=True)
    mask = df['income'].astype(str)=='-'
    df.drop(df[mask].index, inplace=True)
    df.drop(df[df['population']<20].index, inplace=True)  # min 20 people for block to be counted

In [43]:
# blocks = pd.read_csv('SeattleCoords.csv',sep=';',compression='gzip')
blocks = df_coords
blocks.head()

,GEOID10,INTPTLAT10,INTPTLON10,UR10,ALAND10,AWATER10
0,530730101003147,+48.8311355,-122.1424270,R,0,7503
1,530730101004003,+48.8324631,-122.1557965,R,0,12106
2,530730101004182,+48.6698268,-122.1447266,R,0,87185
3,530730110001018,+48.9779153,-123.0668955,R,0,120126
4,530739400001027,+48.7731088,-122.5998590,R,0,81726


In [45]:
# blocks.drop('Unnamed: 0', axis=1, inplace=True)
blocks.columns = ['GEO.id2','latitude','longitude','urban','landarea','waterarea']
blocks['blockgroup'] = blocks['GEO.id2'].map(cut)
blocks['GEO.id2'] = blocks['GEO.id2'].map(int)
blocks = blocks.set_index('GEO.id2')
blocks.sort_index(inplace=True)
blocks.head()

,latitude,longitude,urban,landarea,waterarea,blockgroup
GEO.id2,,,,,,
530019501001000,+47.2512419,-117.9624855,R,570258,0,530019501001
530019501001001,+47.2440890,-118.0011761,R,16110504,178899,530019501001
530019501001002,+47.2227312,-118.0132900,R,0,58282,530019501001
530019501001003,+47.2531211,-117.9782793,R,0,274966,530019501001
530019501001004,+47.2397419,-117.9780911,R,0,477164,530019501001


In [46]:
blocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 195574 entries, 530019501001000 to 530779400062044
Data columns (total 6 columns):
latitude      195574 non-null object
longitude     195574 non-null object
urban         195574 non-null object
landarea      195574 non-null int64
waterarea     195574 non-null int64
blockgroup    195574 non-null int64
dtypes: int64(3), object(3)
memory usage: 10.4+ MB


In [47]:
blocks = blocks.join(populations[['income','population']], how='inner')
blocks.shape

(195574, 8)

In [48]:
blocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 195574 entries, 530019501001000 to 530779400062044
Data columns (total 8 columns):
latitude      195574 non-null object
longitude     195574 non-null object
urban         195574 non-null object
landarea      195574 non-null int64
waterarea     195574 non-null int64
blockgroup    195574 non-null int64
income        195489 non-null float64
population    195574 non-null int64
dtypes: float64(1), int64(4), object(3)
memory usage: 13.4+ MB


In [49]:
blocksdropna(blocks)
blocks.shape

(19356, 8)

In [50]:
blocks[(blocks['population']!=0) & (blocks['income']==0)] # must drop as don't have y target (income)

,latitude,longitude,urban,landarea,waterarea,blockgroup,income,population
GEO.id2,,,,,,,,


In [51]:
testlat, testlong  # used as center of Seattle

(47.592989, -122.336087)

In [52]:
blocks['distance_from_city'] = blocks.apply(
    (lambda row : calcDist(row['latitude'],row['longitude'],testlat, testlong)),         
    axis=1)

In [53]:
blocks = blocks[blocks['distance_from_city'] < (maxradius - 1000)]
blocks.shape

(1976, 9)

In [54]:
blocks = blocks.reindex(['distance_from_city','urban','latitude','longitude','blockgroup','income',
                'blockgrouppop','population','landarea','waterarea'], axis=1)
blocks.head()

,distance_from_city,urban,latitude,longitude,blockgroup,income,blockgrouppop,population,landarea,waterarea
GEO.id2,,,,,,,,,,
530330058022019,4980.066227,U,+47.6309699,-122.3711993,530330058022,50272.0,NaN,256,298132,0
530330062001053,4944.954575,U,+47.6281770,-122.2958621,530330062001,75693.0,NaN,74,135167,0
530330062003002,4858.225523,U,+47.6332217,-122.3108745,530330062003,110602.0,NaN,43,9923,0
530330062003004,4838.755140,U,+47.6332255,-122.3115595,530330062003,110602.0,NaN,37,10647,0
530330062003007,4822.522346,U,+47.6332461,-122.3122076,530330062003,110602.0,NaN,70,8804,0


In [55]:
blocks[blocks['distance_from_city'] < (maxradius - 1000)][['population']].sum()

population    172475
dtype: int64

In [56]:
def bootstrapBlocks(blocksdf) :  # changed add block rows to per 100 from 200
    df = blocksdf.copy()
    print(df.index)
    for index in blocksdf[blocksdf['population']>100].index : 
        iterpop = int(np.ceil(blocksdf.loc[index,'population'] / 100))
        lis = [pd.Series(data=blocksdf.loc[index].values, name=index+(10**15*i),
                        index=blocksdf.columns) for i in range(1,iterpop)]
#         print(len(lis))
        df = df.append(lis) # more efficient and faster to append all at once
    return df

In [57]:
blocks = bootstrapBlocks(blocks)
blocks.shape

Int64Index([530330058022019, 530330062001053, 530330062003002, 530330062003004,
            530330062003007, 530330062003008, 530330062003019, 530330063004021,
            530330063004027, 530330064001003,
            ...
            530330108003010, 530330108003011, 530330108003012, 530330108003013,
            530330108003014, 530330108003015, 530330108003016, 530330109002059,
            530330109002060, 530330109002074],
           dtype='int64', name='GEO.id2', length=1976)


(2747, 10)

In [58]:
blocks.to_csv('SeattleTrainRowCoords.csv',
          sep=';',
          header=True,
          index=True,
          compression='gzip')

## Vectorize Businesses and Assemble X_train

### Construct Category Corpus with Aliases

In [59]:
lat1000m = -115.161605-(-115.172713)

def makecorpus(df, lat, long) :
    distances = calcDistancesV2(df, lat, long)
    distances = pd.DataFrame(data=distances, index=distances.index, columns=['distances'])
    distances = distances.join(df['alias_cats']) #['alias'] = df.loc[distances.index, 'alias']
    print(distances.shape)
    temp = distances.apply(addproximity, axis=1)
    return ' '.join(temp)

def addproximity(dfrow) :
    w = ('near' if (dfrow.loc['distances'] <= 500) else 'far')
    temp = dfrow.loc['alias_cats'].split(' ')
    temp = list(map((lambda x : (x + '_' + w)), temp))
    return ' '.join(temp)

def calcDistancesV2(multiindexdf, lat, long) :
    minlat = lat - lat1000m
    maxlat = lat + lat1000m
    minlong = long - lat1000m
    maxlong = long + lat1000m
#     multiindexdf = multiindexdf.sort_index(level = [0,1])
    idx = pd.IndexSlice
    df = multiindexdf.loc[idx[minlat:maxlat, minlong:maxlong], ['latitude','longitude']]
    distances = pd.Series(index=df.index)
    for index in df.index :
        try :
            distances[index] = calcDist(lat, long, df.loc[index,'latitude'].values[0], 
                                    df.loc[index,'longitude'].values[0])
        except : # catch very rare attribute error to fill in rest of rows
            distances[index] = 10000 # if calcDist gives error, don't include this business
    distances = distances[distances<1000]
    return distances

def make_X_train(coorddf, businessesdf) :
    for index in coorddf.index :
            coorddf.loc[index,'corpus'] = makecorpus(businessesdf, 
                                            coorddf.loc[index,'latitude'], 
                                            coorddf.loc[index,'longitude'])

In [60]:
df = pd.read_csv('SeattleBusinessesFinal.csv',sep=';',compression='gzip',
                 index_col=['lat_index.1','long_index.1'])
df.head()

lat_index  long_index  Unnamed: 0  \
lat_index.1 long_index.1                                      
47.539      -122.3414        47.539   -122.3414       20055   
47.540      -122.3383        47.540   -122.3383       10220   
            -122.3228        47.540   -122.3228       14764   
47.541      -122.3377        47.541   -122.3377       21396   
            -122.3277        47.541   -122.3277        1740   

                                                              alias  \
lat_index.1 long_index.1                                              
47.539      -122.3414     pacific-plumbing-supply-company-seattle-2   
47.540      -122.3383                  dandylines-show-room-seattle   
            -122.3228               calozzis-cheesesteaks-seattle-3   
47.541      -122.3377                   barron-collection-seattle-2   
            -122.3277                    alpine-auto-center-seattle   

                                                                 categories  \
lat_index.1 long_index.1                                                      
47.539      -122.3414          [{'alias': 'plumbing', 'title': 'Plumbing'}]   
47.540      -122.3383     [{'alias': 'homedecor', 'title': 'Home Decor'}...   
            -122.3228     [{'alias': 'cheesesteaks', 'title': 'Cheeseste...   
47.541      -122.3377       [{'alias': 'giftshops', 'title': 'Gift Shops'}]   
            -122.3277     [{'alias': 'bodyshops', 'title': 'Body Shops'}...   

                                                                coordinates  \
lat_index.1 long_index.1                                                      
47.539      -122.3414       {'latitude': 47.53926, 'longitude': -122.34143}   
47.540      -122.3383     {'latitude': 47.5404648292527, 'longitude': -1...   
            -122.3228     {'latitude': 47.540249, 'longitude': -122.3228...   
47.541      -122.3377     {'latitude': 47.5410907, 'longitude': -122.337...   
            -122.3277     {'latitude': 47.5410474, 'longitude': -122.327...   

                             distance   latitude   longitude main_category  \
lat_index.1 long_index.1                                                     
47.539      -122.3414     5987.205780  47.539260 -122.341430  homeservices   
47.540      -122.3383     5842.077766  47.540465 -122.338285  professional   
            -122.3228     5947.788414  47.540249 -122.322845   restaurants   
47.541      -122.3377     5771.500750  47.541091 -122.337745      shopping   
            -122.3277     5809.509324  47.541047 -122.327680    automotive   

                                                     name  price  rating  \
lat_index.1 long_index.1                                                   
47.539      -122.3414     Pacific Plumbing Supply Company      0     4.0   
47.540      -122.3383                DandyLines Show Room      1     5.0   
            -122.3228              Calozzi's Cheesesteaks      2     4.5   
47.541      -122.3377                   Barron Collection      0     5.0   
            -122.3277                  Alpine Auto Center      0     4.0   

                          review_count    distance2  \
lat_index.1 long_index.1                              
47.539      -122.3414                5  5987.205780   
47.540      -122.3383                1  5842.077766   
            -122.3228              234  5947.788414   
47.541      -122.3377                1  5771.500750   
            -122.3277               16  5809.509324   

                                                            alias_cats  
lat_index.1 long_index.1                                                
47.539      -122.3414                        0_homeservices 0_plumbing  
47.540      -122.3383         1_professional 1_homedecor 1_accessories  
            -122.3228                       2_restaurants 2_sandwiches  
47.541      -122.3377                                       0_shopping  
            -122.3277     0_automotive 0_bodyshops 0_auto 0_autorepair

In [61]:
df.describe()

,lat_index,long_index,Unnamed: 0,distance,latitude,longitude,price,rating,review_count,distance2
count,12516.000000,12516.000000,12516.000000,12516.000000,12516.000000,12516.000000,12516.000000,12516.000000,12516.000000,12516.000000
mean,47.604169,-122.334535,7725.405002,2932.728412,47.604174,-122.334538,0.587967,3.912033,53.485459,2932.728412
std,0.022466,0.022057,5348.637234,1397.593555,0.022472,0.022057,0.952829,1.161616,193.667140,1397.593555
min,47.539000,-122.412700,0.000000,142.573040,47.539260,-122.412724,0.000000,0.000000,0.000000,142.573040
25%,47.598000,-122.342000,3456.750000,1888.690672,47.598480,-122.341950,0.000000,3.500000,2.000000,1888.690672
50%,47.610000,-122.333700,6848.500000,2626.398276,47.609999,-122.333683,0.000000,4.000000,6.000000,2626.398276
75%,47.617000,-122.323200,11229.250000,3996.003484,47.617296,-122.323159,1.000000,5.000000,30.000000,3996.003484
max,47.646000,-122.267100,22639.000000,5999.131624,47.646366,-122.267054,4.000000,5.000000,6441.000000,5999.131624


In [62]:
blocks = pd.read_csv('SeattleTrainRowCoords.csv',
          sep=';', compression='gzip', index_col='Unnamed: 0')
blocks.head()

,distance_from_city,urban,latitude,longitude,blockgroup,income,blockgrouppop,population,landarea,waterarea
530330058022019,4980.066227,U,47.630970,-122.371199,530330058022,50272.0,NaN,256,298132,0
530330062001053,4944.954575,U,47.628177,-122.295862,530330062001,75693.0,NaN,74,135167,0
530330062003002,4858.225523,U,47.633222,-122.310875,530330062003,110602.0,NaN,43,9923,0
530330062003004,4838.755140,U,47.633226,-122.311560,530330062003,110602.0,NaN,37,10647,0
530330062003007,4822.522346,U,47.633246,-122.312208,530330062003,110602.0,NaN,70,8804,0


In [63]:
blocks.describe()

,distance_from_city,latitude,longitude,blockgroup,income,blockgrouppop,population,landarea,waterarea
count,2747.000000,2747.000000,2747.000000,2.747000e+03,2747.000000,0.0,2747.000000,2747.000000,2747.000000
mean,3362.345483,47.602146,-122.326458,5.303301e+11,57966.046232,NaN,142.472151,18873.757918,21.972697
std,1027.143698,0.022245,0.028782,1.184066e+04,24666.958432,NaN,181.049008,35213.060607,1151.629301
min,413.685406,47.549469,-122.400090,5.303301e+11,8573.000000,NaN,20.000000,1012.000000,0.000000
25%,2647.113188,47.583309,-122.346395,5.303301e+11,42373.000000,NaN,46.000000,9173.000000,0.000000
50%,3484.033740,47.608504,-122.320085,5.303301e+11,55301.000000,NaN,86.000000,12491.000000,0.000000
75%,4191.977459,47.619366,-122.304370,5.303301e+11,70186.000000,NaN,171.000000,16625.000000,0.000000
max,4999.365340,47.637551,-122.280177,5.303301e+11,195814.000000,NaN,1662.000000,819365.000000,60359.000000


In [64]:
make_X_train(blocks, df)

(769, 2)
(481, 2)
(419, 2)
(387, 2)
(387, 2)
(239, 2)
(312, 2)
(457, 2)
(437, 2)
(718, 2)
(711, 2)
(724, 2)
(709, 2)
(717, 2)
(738, 2)
(736, 2)
(739, 2)
(748, 2)
(738, 2)
(458, 2)
(744, 2)
(757, 2)
(1096, 2)
(1007, 2)
(741, 2)
(754, 2)
(448, 2)
(1103, 2)
(765, 2)
(419, 2)
(620, 2)
(672, 2)
(1699, 2)
(996, 2)
(761, 2)
(777, 2)
(1585, 2)
(1641, 2)
(1924, 2)
(1825, 2)
(840, 2)
(476, 2)
(367, 2)
(385, 2)
(415, 2)
(755, 2)
(691, 2)
(673, 2)
(617, 2)
(599, 2)
(910, 2)
(971, 2)
(978, 2)
(937, 2)
(800, 2)
(988, 2)
(1026, 2)
(1012, 2)
(2203, 2)
(2160, 2)
(2163, 2)
(1063, 2)
(1980, 2)
(1274, 2)
(1359, 2)
(1355, 2)
(1635, 2)
(1613, 2)
(1736, 2)
(1778, 2)
(3590, 2)
(3450, 2)
(3594, 2)
(3520, 2)
(5173, 2)
(4940, 2)
(4831, 2)
(4868, 2)
(4654, 2)
(4957, 2)
(1941, 2)
(2158, 2)
(3502, 2)
(3580, 2)
(3326, 2)
(3541, 2)
(3662, 2)
(3482, 2)
(3471, 2)
(3605, 2)
(3405, 2)
(3389, 2)
(3354, 2)
(1958, 2)
(2008, 2)
(3456, 2)
(3369, 2)
(3312, 2)
(3346, 2)
(2161, 2)
(2287, 2)
(2069, 2)
(2022, 2)
(1433, 2)
(1786, 2

(83567, 2)
(117025, 2)
(106457, 2)
(110613, 2)
(108154, 2)
(118460, 2)
(115406, 2)
(128746, 2)
(128891, 2)
(129005, 2)
(127165, 2)
(128128, 2)
(127300, 2)
(127341, 2)
(127479, 2)
(126883, 2)
(127086, 2)
(125891, 2)
(129480, 2)
(125417, 2)
(125664, 2)
(119938, 2)
(117023, 2)
(118907, 2)
(125409, 2)
(127024, 2)
(126847, 2)
(106073, 2)
(110547, 2)
(115482, 2)
(115201, 2)
(125892, 2)
(127446, 2)
(21380, 2)
(80467, 2)
(81665, 2)
(82407, 2)
(81223, 2)
(21946, 2)
(18443, 2)
(18680, 2)
(36839, 2)
(98553, 2)
(100233, 2)
(38319, 2)
(100225, 2)
(114133, 2)
(117030, 2)
(101680, 2)
(76163, 2)
(52167, 2)
(114705, 2)
(116204, 2)
(88181, 2)
(88107, 2)
(89350, 2)
(106635, 2)
(105915, 2)
(87939, 2)
(88079, 2)
(105027, 2)
(110591, 2)
(110865, 2)
(121818, 2)
(126672, 2)
(118293, 2)
(119471, 2)
(119908, 2)
(117502, 2)
(118700, 2)
(78668, 2)
(49625, 2)
(55852, 2)
(53829, 2)
(50878, 2)
(48510, 2)
(47770, 2)
(48084, 2)
(45955, 2)
(56070, 2)
(115560, 2)
(12614, 2)
(13839, 2)
(14108, 2)
(13977, 2)
(13819, 2)
(1

(140, 2)
(140, 2)
(140, 2)
(144, 2)
(756, 2)
(763, 2)
(760, 2)
(764, 2)
(991, 2)
(155, 2)
(155, 2)
(154, 2)
(157, 2)
(1130, 2)
(1138, 2)
(1278, 2)
(363, 2)
(432, 2)
(424, 2)
(1922, 2)
(675, 2)
(402, 2)
(971, 2)
(986, 2)
(758, 2)
(1028, 2)
(1105, 2)
(1111, 2)
(1120, 2)
(1094, 2)
(847, 2)
(280, 2)
(845, 2)
(711, 2)
(678, 2)
(240, 2)
(201, 2)
(203, 2)
(192, 2)
(193, 2)
(193, 2)
(168, 2)
(159, 2)
(136, 2)
(161, 2)
(176, 2)
(137, 2)
(134, 2)
(81, 2)
(301, 2)
(291, 2)
(293, 2)
(279, 2)
(240, 2)
(107, 2)
(115, 2)
(86, 2)
(95, 2)
(119, 2)
(102, 2)
(122, 2)
(100, 2)
(179, 2)
(118, 2)
(89, 2)
(88, 2)
(96, 2)
(144, 2)
(185, 2)
(160, 2)
(212, 2)
(238, 2)
(330, 2)
(405, 2)
(216, 2)
(393, 2)
(920, 2)
(945, 2)
(940, 2)
(880, 2)
(846, 2)
(873, 2)
(931, 2)
(928, 2)
(873, 2)
(844, 2)
(843, 2)
(869, 2)
(928, 2)
(315, 2)
(303, 2)
(313, 2)
(322, 2)
(347, 2)
(764, 2)
(854, 2)
(964, 2)
(920, 2)
(896, 2)
(793, 2)
(875, 2)
(780, 2)
(781, 2)
(747, 2)
(868, 2)
(703, 2)
(683, 2)
(798, 2)
(822, 2)
(816, 2)
(919, 2

(50878, 2)
(50878, 2)
(48510, 2)
(48084, 2)
(45955, 2)
(56070, 2)
(56070, 2)
(56070, 2)
(56070, 2)
(56070, 2)
(115560, 2)
(115560, 2)
(115560, 2)
(115560, 2)
(115560, 2)
(115560, 2)
(115560, 2)
(115560, 2)
(115560, 2)
(115560, 2)
(115560, 2)
(115560, 2)
(115560, 2)
(115560, 2)
(115560, 2)
(115560, 2)
(12614, 2)
(13839, 2)
(12609, 2)
(13939, 2)
(13853, 2)
(15166, 2)
(14733, 2)
(14733, 2)
(14733, 2)
(27503, 2)
(27503, 2)
(27503, 2)
(27503, 2)
(27503, 2)
(27503, 2)
(27503, 2)
(27503, 2)
(27503, 2)
(27503, 2)
(27503, 2)
(27503, 2)
(27293, 2)
(49161, 2)
(49161, 2)
(26394, 2)
(26394, 2)
(13639, 2)
(14545, 2)
(11719, 2)
(11719, 2)
(11719, 2)
(44612, 2)
(6380, 2)
(1361, 2)
(1080, 2)
(960, 2)
(5986, 2)
(1307, 2)
(1199, 2)
(403, 2)
(147, 2)
(147, 2)
(147, 2)
(225, 2)
(351, 2)
(250, 2)
(250, 2)
(969, 2)
(978, 2)
(888, 2)
(808, 2)
(323, 2)
(323, 2)
(404, 2)
(1207, 2)
(1207, 2)
(1678, 2)
(2395, 2)
(1276, 2)
(1194, 2)
(1207, 2)
(20445, 2)
(22319, 2)
(22319, 2)
(22319, 2)
(22494, 2)
(15027, 2)
(1814,

In [65]:
blocks.to_csv('CorpusSeattleTrainRowCoords.csv',sep=';',compression='gzip')
blocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2747 entries, 530330058022019 to 1530330108003016
Data columns (total 11 columns):
distance_from_city    2747 non-null float64
urban                 2747 non-null object
latitude              2747 non-null float64
longitude             2747 non-null float64
blockgroup            2747 non-null int64
income                2747 non-null float64
blockgrouppop         0 non-null float64
population            2747 non-null int64
landarea              2747 non-null int64
waterarea             2747 non-null int64
corpus                2747 non-null object
dtypes: float64(5), int64(4), object(2)
memory usage: 337.5+ KB


In [66]:
X_features = joblib.load('X_features.joblib')

vect = CountVectorizer(dtype='uint16',min_df=1, vocabulary=X_features,
                       decode_error='ignore', lowercase=True) 

X_sparse = vect.transform(blocks['corpus'])

dfvect = pd.DataFrame(X_sparse.toarray(),dtype='uint16',columns=X_features)


In [68]:
joblib.dump(dfvect, 'X_trainSeattle.joblib', compress=9)

['X_trainSeattle.joblib']

In [69]:
y_trainSeattle = blocks['income']

In [70]:
joblib.dump(y_trainSeattle, 'y_trainSeattle.joblib', compress=9)

['y_trainSeattle.joblib']